**Processing METADATA File**

*1- Data Exploration of MetaData File*

In [22]:
import json
import gzip

def inspect_json_sample(gz_path, sample_size=5):
    sample_data = []

    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f, 1):
            if i > sample_size:
                break  # Read only the first 'sample_size' lines
            try:
                data = json.loads(line.strip())
                sample_data.append(data)
            except json.JSONDecodeError:
                continue  # Skip bad lines

    # Print sample data and the keys (columns)
    for idx, data in enumerate(sample_data):
        print(f"\nSample {idx + 1}:")
        print(json.dumps(data, indent=2))
        print("Keys:", list(data.keys()))

gz_file = r'C:\Users\LAPTOP\Downloads\meta_Electronics.jsonl.gz'
inspect_json_sample(gz_file)



Sample 1:
{
  "main_category": "All Electronics",
  "title": "FS-1051 FATSHARK TELEPORTER V3 HEADSET",
  "average_rating": 3.5,
  "rating_number": 6,
  "features": [],
  "description": [
    "Teleporter V3 The \u201cTeleporter V3\u201d kit sets a new level of value in the FPV world with Fat Shark renowned performance and quality. The fun of FPV is experienced firsthand through the large screen FPV headset with integrated NexwaveRF receiver technology while simultaneously recording onboard HD footage with the included \u201cPilotHD\u201d camera. The \u201cTeleporter V3\u201d kit comes complete with everything you need to step into the cockpit of your FPV vehicle. We\u2019ve included our powerful 250mW 5.8Ghz transmitter, 25 degree FOV headset (largest QVGA display available), the brand new \u201cPilotHD\u201d camera with live AV out and all the cables, antennas and connectors needed."
  ],
  "price": null,
  "images": [
    {
      "thumb": "https://m.media-amazon.com/images/I/41qrX56l

*2- Extracting Samples for one year- 2021 into CSV file*

In [25]:
import json
import gzip
from collections import defaultdict
import re

def count_samples_by_year(gz_path):
    year_counts = defaultdict(int)

    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line.strip())

                # Check if 'details' field and 'Date First Available' exist
                details = data.get('details', {})
                date_str = details.get('Date First Available')

                if date_str:
                    # Extract year using regex
                    match = re.search(r'\b(\d{4})\b', date_str)
                    if match:
                        year = match.group(1)
                        year_counts[year] += 1

            except json.JSONDecodeError:
                continue  # Skip malformed lines

    # Print results
    for year, count in sorted(year_counts.items()):
        print(f"{year}: {count} samples")

# Path to your file
gz_file = r'C:\Users\LAPTOP\Downloads\meta_Electronics.jsonl.gz'
count_samples_by_year(gz_file)

1931: 8 samples
1955: 5 samples
1957: 2 samples
1969: 2 samples
1970: 2 samples
1973: 1434 samples
1977: 1 samples
1979: 2 samples
1982: 1 samples
1988: 1 samples
1989: 3 samples
1990: 6 samples
1994: 1 samples
1995: 5 samples
1996: 2 samples
1997: 1 samples
1998: 2 samples
1999: 2067 samples
2000: 1091 samples
2001: 8392 samples
2002: 4214 samples
2003: 13047 samples
2004: 21630 samples
2005: 10978 samples
2006: 12859 samples
2007: 13515 samples
2008: 17742 samples
2009: 26659 samples
2010: 42508 samples
2011: 49412 samples
2012: 73694 samples
2013: 80598 samples
2014: 84904 samples
2015: 97247 samples
2016: 130006 samples
2017: 116690 samples
2018: 130253 samples
2019: 175473 samples
2020: 145537 samples
2021: 133865 samples
2022: 95482 samples
2023: 20406 samples


In [38]:
import json
import gzip
import csv
import re

def extract_2021_samples_to_csv(gz_path, output_csv):
    writer = None
    fieldnames = set()
    rows = []

    with gzip.open(gz_path, 'rt', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line.strip())
                details = data.get('details', {})
                date_str = details.get('Date First Available')

                if date_str and re.search(r'\b2021\b', date_str):
                    # Flatten details first, then update with top-level keys (asin preserved)
                    flat_data = {**details, **data}
                    rows.append(flat_data)
                    fieldnames.update(flat_data.keys())
            except json.JSONDecodeError:
                continue

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=list(fieldnames))
        writer.writeheader()
        for row in rows:
            writer.writerow({key: row.get(key, '') for key in fieldnames})

# Paths
gz_file = r'C:\Users\LAPTOP\Downloads\meta_Electronics.jsonl.gz'
output_csv = r'D:\MetadataElectronics_2021.csv'

extract_2021_samples_to_csv(gz_file, output_csv)


In [30]:
print("ASIN" in df1.columns)      # Returns True if 'asin' is present
print(df1.columns[df1.columns.str.contains("asin", case=False)])  # Shows any columns with 'asin' in name, we will later merge using asin


False
Index(['parent_asin'], dtype='object')


In [31]:
print([col for col in df1.columns if 'asin' in col.lower() or 'id' in col.lower()])

['Battery Cell Composition Sealed Lead Acid', 'LAN Port Bandwidth', 'Video Input', 'Inside Pockets', 'Video Capture Resolution', 'Material Vinyl, Polyvinyl Chloride', 'Video Output Interface', 'Chamber Width', 'Video Standard', 'videos', 'Video Capture Format', 'Liquid Volume', 'Material Polyvinyl Chloride', 'parent_asin', 'Style Mid-Century Modern', 'Video encoding', 'Band Width', 'Width (inches)', 'Internet service provider', 'Video Output Resolution', 'Media Format Digital Video', 'Base Width', 'Input Video Compatability', 'Caller Identification', 'Number of Dividers', 'Operating Humidity', 'Effective Video Resolution', 'Assembled Width']


*3- Exploring CSV files*

In [ ]:
import pandas as pd

# Path to your saved CSV
csv_path = r'D:\MetadataElectronics_2021.csv'

# Load it into a DataFrame
df1 = pd.read_csv(csv_path, low_memory=False)

In [4]:
df1.head()

,Unnamed: 0,"Switch Type Toggle, Touch Switch",Networking Feature,"Mounting Type Wall Mount, found in image",Is framed?,"Surface Recommendation Window,, wall,, car,, laptop",GPU,Write Speed,author,Tension Level,...,Print media,Fabric Type,Maximum Frequency,Has Nonstick Coating,"Mounting Type Desk Fan,Tabletop,Neck Fan,Freestanding","Mounting Type Clip On,Freestanding,Neck Fan",Leg Diameter,Maximum Weight Recommendation,Mixer Channel Quantity,"Surface Recommendation Floor, Wall"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df1.columns

Index(['Unnamed: 0', 'Switch Type Toggle, Touch Switch', 'Networking Feature',
       'Mounting Type Wall Mount, found in image', 'Is framed?',
       'Surface Recommendation Window,, wall,, car,, laptop', 'GPU',
       'Write Speed', 'author', 'Tension Level',
       ...
       'Print media', 'Fabric Type', 'Maximum Frequency',
       'Has Nonstick Coating',
       'Mounting Type Desk Fan,Tabletop,Neck Fan,Freestanding',
       'Mounting Type Clip On,Freestanding,Neck Fan', 'Leg Diameter',
       'Maximum Weight Recommendation', 'Mixer Channel Quantity',
       'Surface Recommendation Floor, Wall'],
      dtype='object', length=1242)

In [15]:
df1.shape

(133865, 1242)

In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133865 entries, 0 to 133864
Columns: 1242 entries, Unnamed: 0 to Surface Recommendation Floor, Wall
dtypes: float64(454), int64(1), object(787)
memory usage: 1.2+ GB


In [33]:
meta_columns_to_keep = [
    'parent_asin',
    'brand',
    'title',
    'price',
    'category',  # Only if exists, or else remove from this list
    'author',
    'GPU',
    'Write Speed',
    'Maximum Frequency',
    'Mounting Type Wall Mount, found in image',
    'Surface Recommendation Window,, wall,, car,, laptop',
    'Is framed?',
    'imageURL'  # If present
]

# As metadata contains too many columns so, Only keep these columns (handle missing ones)
df_meta_cleaned = df1[[col for col in meta_columns_to_keep if col in df1.columns]]


In [34]:
df_meta_cleaned.columns

Index(['parent_asin', 'title', 'price', 'author', 'GPU', 'Write Speed',
       'Maximum Frequency', 'Mounting Type Wall Mount, found in image',
       'Surface Recommendation Window,, wall,, car,, laptop', 'Is framed?'],
      dtype='object')

**Processing REVIEW files**

*1- Data Exploration of Review File*

In [12]:
def inspect_raw_lines(file_path, num_lines=5):
    with open(file_path, 'r', encoding='utf-8') as f:
        for i in range(num_lines):
            line = f.readline()
            print(f"\nRaw line {i+1}:")
            print(line[:500])  # print first 500 chars just in case it's long

# Check first few lines in raw format
file_path = r'C:\Users\LAPTOP\Downloads\amazon_reviews_Electronics_2020-2022_max100000.json'
inspect_raw_lines(file_path)


Raw line 1:
[{"rating": 3.0, "title": "Smells like gasoline! Going back!", "text": "First & most offensive: they reek of gasoline so if you are sensitive/allergic to petroleum products like I am you will want to pass on these.  Second: the phone adapter is useless as-is. Mine was not drilled far enough to be able to tighten it into place for my iPhone 12 max. It just slipped & slid all over. Stupid me putting the adapter together first without picking up the binoculars to smell them bc I wasted 15 minutes t

Raw line 2:
{"rating": 1.0, "title": "The clipse don't grip", "text": "The bands would not stay attached to my charge 3. If I swing my arm or my shirt sleeve bumped it the band would detach. I would not recommend this even though they look cool and it's nice to have a metal bracelet I need to work on the clip which connects it to the actual Fitbit device.", "images": [], "asin": "B07YYD5TNH", "parent_asin": "B07YYD5TNH", "user_id": "AFANVB6MPHJTCTFOVIEBKLWZ2GVA", "timestamp": 1578

*2- Extracting Samples for one year 2021 into CSV file*

In [13]:
import json
import pandas as pd
from datetime import datetime

def filter_reviews_2021(file_path, output_csv):
    # Step 1: Load full JSON array
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)  # This loads the entire array

    print(f"Total reviews loaded: {len(data)}")

    filtered = []

    # Step 2: Iterate and filter based on timestamp
    for item in data:
        timestamp = item.get('timestamp')
        if timestamp:
            try:
                dt = datetime.fromtimestamp(timestamp / 1000)  # Convert ms to datetime
                if dt.year == 2021:
                    item['readable_date'] = dt.strftime('%Y-%m-%d')
                    filtered.append(item)
            except Exception as e:
                continue  # If timestamp is malformed, skip

    # Step 3: Convert to DataFrame and save
    if filtered:
        df = pd.DataFrame(filtered)
        df.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"✅ Saved {len(filtered)} reviews from 2021 to: {output_csv}")
    else:
        print("⚠️ No reviews from 2021 were found.")

# Run it
file_path = r'C:\Users\LAPTOP\Downloads\amazon_reviews_Electronics_2020-2022_max100000.json'
output_csv = r'D:\amazon_reviews_2021.csv'
filter_reviews_2021(file_path, output_csv)

Total reviews loaded: 100000
✅ Saved 34608 reviews from 2021 to: D:\amazon_reviews_2021.csv


*3- Exploring CSV files*

In [22]:
import pandas as pd
# Path to your saved CSV
csv_path = r'D:\amazon_reviews_2021.csv'
# Load it into a DataFrame
df_review = pd.read_csv(csv_path)

In [23]:
df_review.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,readable_date
0,4.0,Sturdy,This may look ugly on a table but It really do...,[],B09FKWPSPH,B0BSCCQLCY,AFFZVSTUS3U2ZD22A2NPZSKOCPGQ,1639078893290,0,False,2021-12-10
1,5.0,Get one,Excellent!! How we've managed with mediocre pr...,[],B079JD7F7G,B0BZJZKYST,AEMJ2EG5ODOCYUTI54NBXZHDJGSQ,1613110535413,1,True,2021-02-12
2,5.0,Excellent speed and performance,Works great for cameras and other non PoE conn...,[],B099PK5NPG,B0B4YTVNFL,AFJBKPK5W56XWSNPQU2WW66ISWYQ,1634228294308,0,False,2021-10-14
3,5.0,Works fine,Good box,[],B01LYW41TH,B01LYW41TH,AEHJJPC3MQJAHAJWBEVIF5EFQQBQ,1612992196552,0,True,2021-02-11
4,5.0,The add-on solution!!,Perfect for anyone with a computer that may la...,[],B09DKNYKCN,B09DKNYKCN,AEIPJBAN7A55Q5DFFPZSR2UV3OKA,1637665455600,1,False,2021-11-23


In [24]:
df_review.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'readable_date'],
      dtype='object')

In [25]:
df_review.shape

(34608, 11)

In [26]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34608 entries, 0 to 34607
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             34608 non-null  float64
 1   title              34586 non-null  object 
 2   text               34594 non-null  object 
 3   images             34608 non-null  object 
 4   asin               34608 non-null  object 
 5   parent_asin        34608 non-null  object 
 6   user_id            34608 non-null  object 
 7   timestamp          34608 non-null  int64  
 8   helpful_vote       34608 non-null  int64  
 9   verified_purchase  34608 non-null  bool   
 10  readable_date      34608 non-null  object 
dtypes: bool(1), float64(1), int64(2), object(7)
memory usage: 2.7+ MB


**MERGE reviews and metadata dataframes**

In [36]:
df_merged = pd.merge(df_review, df_meta_cleaned, on='parent_asin', how='inner')

In [37]:
df_merged.columns

Index(['rating', 'title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'readable_date',
       'title_y', 'price', 'author', 'GPU', 'Write Speed', 'Maximum Frequency',
       'Mounting Type Wall Mount, found in image',
       'Surface Recommendation Window,, wall,, car,, laptop', 'Is framed?'],
      dtype='object')

In [41]:
df_merged.to_csv(r'D:\merged_amazon_data_2021.csv', index=False)

In [6]:
import pandas as pd

# Load your merged file (replace with your actual merged file if needed)
df = pd.read_csv(r'D:\merged_amazon_data_2021.csv')

# 1. Drop rows with any missing values
df_cleaned = df.dropna(subset=['text', 'title_x'])

# 2. Convert `readable_date` to datetime forma
df_cleaned['readable_date'] = pd.to_datetime(df_cleaned['readable_date'], errors='coerce')

# Optional: Drop rows where date conversion failed
df_cleaned = df_cleaned.dropna(subset=['readable_date'])

# 3. Add a new column for Year
df_cleaned['year'] = 2021  # Since your dataset is already from 2021

# 4. Save cleaned dataframe to CSV for Tableau
df_cleaned.to_csv(r'D:\Preprocessed_amazon_2021_electronics.csv', index=False)

print("✅ Cleaned file saved as final_amazon_2021_electronics.csv")


✅ Cleaned file saved as final_amazon_2021_electronics.csv


C:\Users\LAPTOP\AppData\Local\Temp\ipykernel_13056\1619324422.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['readable_date'] = pd.to_datetime(df_cleaned['readable_date'], errors='coerce')


In [7]:
df_cleaned.columns

Index(['rating', 'title_x', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase', 'readable_date',
       'title_y', 'price', 'author', 'GPU', 'Write Speed', 'Maximum Frequency',
       'Mounting Type Wall Mount, found in image',
       'Surface Recommendation Window,, wall,, car,, laptop', 'Is framed?',
       'year'],
      dtype='object')

In [8]:
df_cleaned.shape

(4412, 21)